<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/options/europeans_americans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.stats import norm


# Black-Scholes valuation for European options

def callBS(s, K, T, sigma, r, q):
    s = s if s != 0 else 1.0e-6
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return np.exp(-q * T) * s * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)

def putBS(s, K, T, sigma, r, q):
    s = s if s != 0 else 1.0e-6
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return np.exp(-r * T) * K * norm.cdf(-d2) - np.exp(-q * T) * s * norm.cdf(-d1)

# Binomial trees valuation for American options

def callAmerican(s, K, T, sigma, r, q, N=40):
    s = s if s != 0 else 1.0e-6
    dt = T / N
    up = np.exp(sigma * np.sqrt(dt))
    down = 1 / up
    prob = (np.exp((r - q) * dt) - down) / (up - down)
    discount = np.exp(-r * dt)
    v = np.zeros(N + 1)
    x = s * up ** N
    v[0] = np.maximum(x - K, 0)
    for i in range(1, N + 1):
        x *= down * down
        v[i] = np.maximum(x - K, 0)
    for n in range(N - 1, -1, -1):
        x = s * up ** n
        v[0] = np.maximum(x - K, discount * (prob * v[0] + (1 - prob) * v[1]))
        for i in range(1, n + 1):
            x *= down * down
            v[i] = np.maximum(
                x - K, discount * (prob * v[i] + (1 - prob) * v[i + 1])
            )
    return v[0]

def putAmerican(s, K, T, sigma, r, q, N=40):
    s = s if s != 0 else 1.0e-6
    dt = T / N
    up = np.exp(sigma * np.sqrt(dt))
    down = 1 / up
    prob = (np.exp((r - q) * dt) - down) / (up - down)
    discount = np.exp(-r * dt)
    v = np.zeros(N + 1)
    x = s * up ** N
    v[0] = np.maximum(K - x, 0)
    for i in range(1, N + 1):
        x *= down * down
        v[i] = np.maximum(K - x, 0)
    for n in range(N - 1, -1, -1):
        x = s * up ** n
        v[0] = np.maximum(K - x, discount * (prob * v[0] + (1 - prob) * v[1]))
        for i in range(1, n + 1):
            x *= down * down
            v[i] = np.maximum(
                K - x, discount * (prob * v[i] + (1 - prob) * v[i + 1])
            )
    return v[0]

In [ ]:
# example parameters

S = 50
K = 55
T = 1
sigma = 0.4
r = 0.02
q = 0.03

print('The value for European call is', callBS(S, K, T, sigma, r, q).round(2), end='.\n')
print('The value for American call is', callAmerican(S, K, T, sigma, r, q).round(2), end='.\n')
print('The value for European call is', putBS(S, K, T, sigma, r, q).round(2), end='.\n')
print('The value for American call is', putAmerican(S, K, T, sigma, r, q).round(2), end='.\n')

The value for European call is 5.7.
The value for American call is 5.78.
The value for European call is 11.09.
The value for American call is 11.13.
